## tesing dataloader V4

new ideas with ordinal c and yc 
has ordinal binary and continous outputs

In [2]:
# Load dependencies
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import os
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split



from utils.graph import *
from utils.loss_ordinal import *
from utils.tram_model_helpers import *
from utils.tram_models import *
from utils.tram_data import *


import torch

if torch.cuda.is_available():
    device = torch.device('cuda')
    print("Train with GPU support.")
else:
    device = torch.device('cpu')
    print("No GPU found, train with CPU support.")

Train with GPU support.


adjustet funcitnos for ordinal outcomes 

dev ordinal

In [3]:
experiment_name = "testing_v4_dataloader"   ## <--- set experiment name
seed=42
np.random.seed(seed)

LOG_DIR="/home/bule/TramDag/dev_experiment_logs"
EXPERIMENT_DIR = os.path.join(LOG_DIR, experiment_name)
DATA_PATH = EXPERIMENT_DIR # <----------- change to different source if needed
CONF_DICT_PATH = os.path.join(EXPERIMENT_DIR, f"configuration.json")

In [ ]:
# TODO make  ordinal variables ned new coding since the can funciton as targets and predictors and can be encoded as contionus and categorical 

datatype= "ordinal_Xc_Yc" # as X its continous aswell as Y continous
datatype= "ordinal_Xc_Yn"
datatype= "ordinal_Xn_Yc"
datatype= "ordinal_Xn_Yn"


In [4]:
df = pd.DataFrame({
    "x1": np.random.normal(loc=0, scale=1, size=1000),
    # "x2": np.random.uniform(low=0, high=10, size=1000),
    "ord_bin1":  np.random.binomial(1, p=0.4, size=1000),
    "ord_multi": np.random.choice([0, 1, 2, 3], size=1000, p=[0.2, 0.3, 0.3, 0.2]),
    "ord_bin2":  np.random.binomial(1, p=0.4, size=1000),

})

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

quantiles = train_df.quantile([0.05, 0.95])
min_vals = quantiles.loc[0.05]
max_vals = quantiles.loc[0.95]



data_type={key:value for key, value in zip(train_df.columns, ['cont']*1+['ord']*2+['ord'])}

configuration_dict=new_conf_dict(experiment_name,EXPERIMENT_DIR,DATA_PATH,LOG_DIR)

In [5]:
def create_levels_dict(df:pd.DataFrame,data_type:dict):
    levels_dict={}
    for key,val in data_type.items():
        if val=='ord':
            check_ordinal_variable_values(df,key)
            levels_dict[key]=len(np.unique(df[key]))
    return levels_dict   

def check_ordinal_variable_values(df, var):
    unique_vals = set(df[var].dropna().unique())
    num_classes = len(unique_vals)

    if num_classes < 2:
        raise ValueError(
            f"Variable '{var}' has fewer than 2 unique values: {unique_vals}. "
            "Ordinal variables must have at least two distinct values."
        )

    if num_classes == 2:
        if unique_vals != {0, 1}:
            raise ValueError(
                f"Variable '{var}' is marked as ordinal with 2 classes, "
                f"but values are {unique_vals}. Please provide binary ordinal variables as 0 and 1."
            )
    else:
        if not all(isinstance(val, (int, np.integer)) for val in unique_vals):
            raise ValueError(
                f"Variable '{var}' contains non-integer values: {unique_vals}. "
                "Multiclass ordinal variables must contain only integer values."
            )
        expected_vals = set(range(num_classes))
        if unique_vals != expected_vals:
            raise ValueError(
                f"Variable '{var}' has values {sorted(unique_vals)}, "
                f"but expected values are {sorted(expected_vals)} (0 to {num_classes - 1}). "
                "Multiclass ordinal variables must be zero-indexed and contiguous."
            )


levels_dict=create_levels_dict(df,data_type)
levels_dict

{'ord_bin1': 2, 'ord_multi': 4, 'ord_bin2': 2}

In [ ]:
columns = list(data_type.keys())
adj_matrix = np.full((len(columns), len(columns)), "0", dtype=object)

for i in range(len(columns)-1):
    adj_matrix[i, -1] = "ls"
            
nn_names_matrix= create_nn_model_names(adj_matrix,data_type)


target_nodes=create_node_dict_v2(adj_matrix, nn_names_matrix, data_type, min_vals, max_vals,levels_dict)
target_nodes

{'x1': {'Modelnr': 0,
  'data_type': 'cont',
  'node_type': 'source',
  'parents': [],
  'parents_datatype': {},
  'transformation_terms_in_h()': {},
  'min': -1.5316123093173848,
  'max': 1.6860177542255808,
  'transformation_term_nn_models_in_h()': {}},
 'ord_bin1': {'Modelnr': 1,
  'data_type': 'ord',
  'levels': 2,
  'node_type': 'source',
  'parents': [],
  'parents_datatype': {},
  'transformation_terms_in_h()': {},
  'min': 0.0,
  'max': 1.0,
  'transformation_term_nn_models_in_h()': {}},
 'ord_multi': {'Modelnr': 2,
  'data_type': 'ord',
  'levels': 4,
  'node_type': 'source',
  'parents': [],
  'parents_datatype': {},
  'transformation_terms_in_h()': {},
  'min': 0.0,
  'max': 3.0,
  'transformation_term_nn_models_in_h()': {}},
 'ord_bin2': {'Modelnr': 3,
  'data_type': 'ord',
  'levels': 2,
  'node_type': 'sink',
  'parents': ['x1', 'ord_bin1', 'ord_multi'],
  'parents_datatype': {'x1': 'cont', 'ord_bin1': 'ord', 'ord_multi': 'ord'},
  'transformation_terms_in_h()': {'x1': 'l

In [1]:
class GenericDataset_v4(Dataset):
    def __init__(
        self,
        df,
        target_col,
        target_nodes=None,
        parents_dataype_dict=None,
        transformation_terms_in_h=None,
        transform=None
    ):
        """
        df: pd.DataFrame
        target_col: str
        target_nodes: dict mapping each node → metadata (including 'data_type')
        parents_dataype_dict: dict var_name → "cont"|"ord"|"other"
        transform: torchvision transform for images
        transformation_terms_in_h: dict for intercept logic
        """
                
        self.df = df.reset_index(drop=True)
        self.target_col = target_col
        self.target_nodes = target_nodes or {}
        self.parents_dataype_dict = parents_dataype_dict or {}
        self.variables = list(self.parents_dataype_dict.keys())
        self.transform = transform
        self.transformation_terms_in_h = transformation_terms_in_h or {}
        
        self.target_is_source= True if target_nodes[self.target_col].get('node_type').lower() == "source" else False
        self.h_needs_simple_intercept=True if all('i' not in str(v) for v in self.transformation_terms_in_h.values()) else False
        self.ordinal_num_classes ={
                        var: self.df[var].nunique() for var in self.variables
                        if "ordinal" in self.parents_dataype_dict[var].lower() and "Xn" in self.parents_dataype_dict[var].lower()
                        }
        
        
        
        self.target_data_type=self.target_nodes[self.target_col].get('data_type').lower()
        self.target_num_classes=self.target_nodes[self.target_col].get('levels') or None # should be none anywasy if levels not exits

        #checks
        self._check_multiclass_variables_of_df()
        
    def _transform_y(self,row):
        #returns continous or onehot encoded target
        if  self.target_data_type=="continous" or "Yc".lower() in self.target_data_type:
                y = torch.tensor(row[self.target_col], dtype=torch.float32)
                return y
        elif self.target_num_classes is not None:
                        raw = row[self.target_col]
                        y_int = int(raw)
                        y = F.one_hot(torch.tensor(y_int, dtype=torch.long), num_classes=self.target_num_classes).float()
                        return y    

        
    def _check_multiclass_variables_of_df(self):
        # checks whether the predictors here called varibles are ordinal and nominally modelled 
        for var in self.variables:
            dtype = self.parents_dataype_dict[var]
            if "ordinal" not in dtype:
                continue
            
            elif "Xn".lower() in dtype.lower():
                unique_vals = set(self.df[var].dropna().unique())
                num_classes = len(unique_vals)

                if not all(isinstance(val, (int, np.integer)) for val in unique_vals):
                    raise ValueError(
                        f"Variable '{var}' contains non-integer values: {unique_vals}. "
                        "Multiclass ordinal variables must contain only integer values."
                    )

                expected_vals = set(range(num_classes))
                if unique_vals != expected_vals:
                    raise ValueError(
                        f"Variable '{var}' has values {sorted(unique_vals)}, "
                        f"but expected values are {sorted(expected_vals)} (0 to {num_classes - 1}). "
                        "Multiclass ordinal variables must be zero-indexed and contiguous."
                )
            else:
                continue
                        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        # ---------------------------------------- SOURCE NODE: no parents → x = [1.0] ---
        if  self.target_is_source:
            x_data = [torch.tensor(1.0)]
            y=self._transform_y(row)
            return tuple(x_data), y

        # ---------------------------------------- if not source prepare data  ---
        x_data = []

        # --- SIMPLE INTERCEPT if needed  first term in x is x = [1.0]---
        if self.h_needs_simple_intercept:
            x_data.append(torch.tensor(1.0))

        # --- BUILD FEATURES ---
        for var in self.variables:
            dtype = self.parents_dataype_dict[var].lower()
            ## Continous  feature
            if dtype == "continous" or "Xc".lower() in dtype:
                x_data.append(torch.tensor(row[var], dtype=torch.float32))
                
            ## Ordinal feature , if it has more thatn 2 classes it uses onehotencodig, if binary use just 0 and 1
            elif "ordinal" in dtype and "Xn".lower() in dtype:
                x_ord = int(row[var])
                var_num_classes = self.ordinal_num_classes[var]
                x_ord_onehot = F.one_hot(torch.tensor(x_ord, dtype=torch.long),num_classes=var_num_classes).float()
                x_data.append(x_ord_onehot)

            else:  # "other"
                img = Image.open(row[var]).convert("RGB")
                if self.transform:
                    img = self.transform(img)
                x_data.append(img)

        # --- BUILD TARGET ---
        y=self._transform_y(row)
            
            
        return tuple(x_data), y



def get_dataloader_v4(node, target_nodes, train_df, val_df, batch_size=32, verbose=False):
    
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor()
    ])

    parents_dataype_dict, transformation_terms_in_h, _ = ordered_parents(node, target_nodes)
    if verbose:
        print(f"Parents dtype: {parents_dataype_dict}")
    train_ds = GenericDataset_v4(train_df,target_col=node,target_nodes=target_nodes,parents_dataype_dict=parents_dataype_dict,transform=transform,transformation_terms_in_h=transformation_terms_in_h)
    val_ds = GenericDataset_v4(val_df,target_col=node,target_nodes=target_nodes,parents_dataype_dict=parents_dataype_dict,transform=transform,transformation_terms_in_h=transformation_terms_in_h)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

    return train_loader, val_loader

NameError: name 'Dataset' is not defined

In [ ]:
node='ord_bin2'

train_loader, val_loader = get_dataloader_v4(node, target_nodes, train_df, val_df, batch_size=1, verbose=True)

x,y =next(iter(train_loader))

print(x)
print(y)

Parents dtype: OrderedDict([('x1', 'cont'), ('ord_bin1', 'ord'), ('ord_multi', 'ord')])
[tensor([1.]), tensor([-0.0634]), tensor([1]), tensor([[0., 0., 0., 1.]])]
tensor([[1., 0.]])


In [7]:
_, ordered_transformation_terms_in_h, _=ordered_parents(node, target_nodes)
int_input, shift_list = preprocess_inputs(x, ordered_transformation_terms_in_h.values(), device=device)


In [8]:
int_input

tensor([[1.]], device='cuda:0')

In [9]:
shift_list

[tensor([[-0.4501]], device='cuda:0'),
 tensor([[1]], device='cuda:0'),
 tensor([[[0., 0., 1., 0.]]], device='cuda:0')]

In [10]:
# TODO for multiclass extend input fdeatures of the nn models to n classes


def get_fully_specified_tram_model_v3(node: str, target_nodes: dict, verbose=True):
    # Source nodes get a simple intercept only
    if target_nodes[node]['node_type'] == 'source':
        # if target node is ordinal we only need c-1 thetas
        if target_nodes[node]['data_type']=='ord':
            nn_int = SimpleIntercept(n_thetas=target_nodes[node]['levels']-1)
        else:    
            nn_int = SimpleIntercept()
            
        model = TramModel(nn_int, None)
        if verbose:
            print("Source → SimpleIntercept only")
        return model

    # Otherwise gather terms and model names
    _, terms_dict, model_names_dict = ordered_parents(node, target_nodes)
    model_dict = merge_transformation_dicts(terms_dict, model_names_dict)

    # Split intercepts vs. shifts
    intercepts_dict = {
        k: v for k, v in model_dict.items()
        if "ci" in v['h_term'] or "si" in v['h_term']
    }
    shifts_dict = {
        k: v for k, v in model_dict.items()
        if "ci" not in v['h_term'] and "si" not in v['h_term']
    }

    ############################## INTERCEPT NETWORKS #####################################################
    intercept_groups = group_by_base(intercepts_dict, prefixes=("ci", "si"))
    if not intercept_groups:
        if target_nodes[node]['data_type']=='ord':
            nn_int = SimpleIntercept(n_thetas=int(target_nodes[node]['levels'])-1)
        else:    
            nn_int = SimpleIntercept()
    else:
        if len(intercept_groups) > 1:
            raise ValueError("Multiple intercept models detected; only one is supported.")
        feats = next(iter(intercept_groups.values()))
        cls_name = feats[0][1]['class_name']
        base = get_base_model_class(cls_name)
        
        if target_nodes[node]['data_type']=='ord':
            nn_int = globals()[base](n_features=len(feats),n_thetas=int(target_nodes[node]['levels'])-1)
            
        else:    
            nn_int = globals()[base](n_features=len(feats))
        
    ############################### SHIFT NETWORKS ########################################################
    shift_groups = group_by_base(shifts_dict, prefixes=("cs", "ls"))
    nn_shifts = []
    for feats in shift_groups.values():
        cls_name = feats[0][1]['class_name']
        base = get_base_model_class(cls_name)
        nn_shifts.append(globals()[base](n_features=len(feats)))

    # Combine into TramModel
    tram_model = TramModel(nn_int, nn_shifts)
    if verbose:
        print("Constructed TRAM model:", tram_model)
    return tram_model

In [20]:
target_nodes

{'x1': {'Modelnr': 0,
  'data_type': 'cont',
  'node_type': 'source',
  'parents': [],
  'parents_datatype': {},
  'transformation_terms_in_h()': {},
  'min': -1.523303818090823,
  'max': 1.7558465123350986,
  'transformation_term_nn_models_in_h()': {}},
 'ord_bin1': {'Modelnr': 1,
  'data_type': 'ord',
  'levels': 2,
  'node_type': 'source',
  'parents': [],
  'parents_datatype': {},
  'transformation_terms_in_h()': {},
  'min': 0.0,
  'max': 1.0,
  'transformation_term_nn_models_in_h()': {}},
 'ord_multi': {'Modelnr': 2,
  'data_type': 'ord',
  'levels': 4,
  'node_type': 'source',
  'parents': [],
  'parents_datatype': {},
  'transformation_terms_in_h()': {},
  'min': 0.0,
  'max': 3.0,
  'transformation_term_nn_models_in_h()': {}},
 'ord_bin2': {'Modelnr': 3,
  'data_type': 'ord',
  'levels': 2,
  'node_type': 'sink',
  'parents': ['x1', 'ord_bin1', 'ord_multi'],
  'parents_datatype': {'x1': 'cont', 'ord_bin1': 'ord', 'ord_multi': 'ord'},
  'transformation_terms_in_h()': {'x1': 'ls

In [ ]:

node='ord_bin2'




# Otherwise gather terms and model names
_, terms_dict, model_names_dict = ordered_parents(node, target_nodes)
model_dict = merge_transformation_dicts(terms_dict, model_names_dict)

# Split intercepts vs. shifts
intercepts_dict = {
    k: v for k, v in model_dict.items()
    if "ci" in v['h_term'] or "si" in v['h_term']
}
shifts_dict = {
    k: v for k, v in model_dict.items()
    if "ci" not in v['h_term'] and "si" not in v['h_term']
}

############################## INTERCEPT NETWORKS #####################################################
intercept_groups = group_by_base(intercepts_dict, prefixes=("ci", "si"))
if not intercept_groups:
    if target_nodes[node]['data_type']=='ord':
        nn_int = SimpleIntercept(n_thetas=int(target_nodes[node]['levels'])-1)
    else:    
        nn_int = SimpleIntercept()

else:
    if len(intercept_groups) > 1:
        raise ValueError("Multiple intercept models detected; only one is supported.")
    
    
    feats = next(iter(intercept_groups.values()))
    
    cls_name = feats[0][1]['class_name']
    base = get_base_model_class(cls_name)
    
    if target_nodes[node]['data_type']=='ord':
        
        
        nn_int = globals()[base](n_features=len(feats),n_thetas=int(target_nodes[node]['levels'])-1)
        
    else:    
        nn_int = globals()[base](n_features=len(feats))
    

############################### SHIFT NETWORKS ########################################################
shift_groups = group_by_base(shifts_dict, prefixes=("cs", "ls"))
print(shift_groups)
nn_shifts = []
for feats in shift_groups.values():
    print(feats)

    cls_name = feats[0][1]['class_name']
    base = get_base_model_class(cls_name)
    nn_shifts.append(globals()[base](n_features=len(feats)))

# Combine into TramModel
tram_model = TramModel(nn_int, nn_shifts)

defaultdict(<class 'list'>, {'x1': [('x1', {'h_term': 'ls', 'class_name': 'LinearShift'})], 'ord_bin1': [('ord_bin1', {'h_term': 'ls', 'class_name': 'LinearShift'})], 'ord_multi': [('ord_multi', {'h_term': 'ls', 'class_name': 'LinearShift'})]})
[('x1', {'h_term': 'ls', 'class_name': 'LinearShift'})]
[('ord_bin1', {'h_term': 'ls', 'class_name': 'LinearShift'})]
[('ord_multi', {'h_term': 'ls', 'class_name': 'LinearShift'})]


In [28]:
tram_model

TramModel(
  (nn_int): SimpleIntercept(
    (fc): Linear(in_features=1, out_features=1, bias=False)
  )
  (nn_shift): ModuleList(
    (0-2): 3 x LinearShift(
      (fc): Linear(in_features=1, out_features=1, bias=False)
    )
  )
)

In [ ]:
levels_dict={'ord_multi':len(np.unique(df['ord_multi'])),'ord_bin':len(np.unique(df['ord_bin']))}  

columns = list(data_type.keys())
adj_matrix = np.full((len(columns), len(columns)), "0", dtype=object)

# Set last column (edges *to* 'target') as "ls", excluding self-loop
for i in range(len(columns)):
    for j in range(len(columns)):
        if j > i :
            adj_matrix[i, j] = "ls"
    
    
nn_names_matrix= create_nn_model_names(adj_matrix,data_type)
nn_names_matrix

In [ ]:
node='target'
tram_model=get_fully_specified_tram_model_v2('target', target_nodes, verbose=True)
tram_model

In [ ]:
epochs =1000
train_val_loop_v3(
            node,
            target_nodes,
            NODE_DIR,
            tram_model,
            train_loader,
            val_loader,
            epochs,
            optimizer,
            use_scheduler=False,
            scheduler=False,
            save_linear_shifts=False,
            verbose=1,
            device=device)


# Evaluate

In [ ]:
## laoding the best model 
MODEL_PATH,LAST_MODEL_PATH,TRAIN_HIST_PATH,VAL_HIST_PATH=model_train_val_paths(NODE_DIR)

if os.path.exists(MODEL_PATH) and os.path.exists(TRAIN_HIST_PATH) and os.path.exists(VAL_HIST_PATH):
    print("Existing model found. Loading weights and history...")
    tram_model.load_state_dict(torch.load(MODEL_PATH))

In [ ]:
## evaluate on testdata
tram_model.eval()

_, ordered_transformation_terms_in_h, _=ordered_parents(node, target_nodes)

min_vals = torch.tensor(target_nodes[node]['min'], dtype=torch.float32).to(device)
max_vals = torch.tensor(target_nodes[node]['max'], dtype=torch.float32).to(device)
min_max = torch.stack([min_vals, max_vals], dim=0)

with torch.no_grad():
    for x, y in val_loader:
        y = y.to(device)
        
        int_input, shift_list = preprocess_inputs(x, ordered_transformation_terms_in_h.values(), device=device)
        y_pred = tram_model(int_input=int_input, shift_input=shift_list)
        # loss = contram_nll(y_pred, y, min_max=min_max)
        pred_labels = get_pdf_ordinal(get_cdf_ordinal(y_pred)).argmax(dim=1)
        true_labels = y.argmax(dim=1)
        accuracy = (pred_labels == true_labels).float().mean().item()

        print(f"Accuracy: {accuracy*100:.1f}%")  # → 100.0%